# Работа с датами и временем

### Конвертирование строковых значений в даты

In [3]:
from sqlite3.dbapi2 import Timestamp

import numpy as np
import pandas as pd
date_strings = np.array(['03-04-2005 11:35 PM',
                         '23-05-2010 12:01 AM',
                         '04-09-2009 09:09 PM'])
[pd.to_datetime(date, format='%d-%m-%Y %I:%M %p') for date in date_strings]

[Timestamp('2005-04-03 23:35:00'),
 Timestamp('2010-05-23 00:01:00'),
 Timestamp('2009-09-04 21:09:00')]

### Обработка часовых поясов

In [4]:
pd.Timestamp('2017-05-01 06:00:00', tz='Europe/London')

Timestamp('2017-05-01 06:00:00+0100', tz='Europe/London')

In [7]:
date = pd.Timestamp('2017-05-01 06:00:00')
date_in_london = date.tz_localize('Europe/London')
date_in_london

Timestamp('2017-05-01 06:00:00+0100', tz='Europe/London')

In [9]:
date_in_london.tz_convert('Africa/Abidjan')

Timestamp('2017-05-01 05:00:00+0000', tz='Africa/Abidjan')

In [11]:
from pytz import  all_timezones
all_timezones[0:3]

['Africa/Abidjan', 'Africa/Accra', 'Africa/Addis_Ababa']

### Выбор дат и времени

In [14]:
dataframe = pd.DataFrame()
dataframe['дата'] = pd.date_range('1/1/2001', periods=100000, freq='h')
dataframe[(dataframe['дата'] > '2002-1-1 01:00:00') &
          (dataframe['дата'] <= '2002-1-1 04:00:00')]

,дата
8762,2002-01-01 02:00:00
8763,2002-01-01 03:00:00
8764,2002-01-01 04:00:00


In [15]:
dataframe = dataframe.set_index(dataframe['дата'])
dataframe.loc['2002-1-1 01:00:00':'2002-1-1 04:00:00']

,дата
дата,
2002-01-01 01:00:00,2002-01-01 01:00:00
2002-01-01 02:00:00,2002-01-01 02:00:00
2002-01-01 03:00:00,2002-01-01 03:00:00
2002-01-01 04:00:00,2002-01-01 04:00:00


### Разбиение данных даты на несколько признаков

In [2]:
import pandas as pd
dataframe = pd.DataFrame()

dataframe['дата'] = pd.date_range('1/1/2001', periods=150, freq='W')
dataframe['год'] = dataframe['дата'].dt.year
dataframe['месяц'] = dataframe['дата'].dt.month
dataframe['день'] = dataframe['дата'].dt.day
dataframe ['час'] = dataframe['дата'].dt.hour
dataframe['минута'] = dataframe['дата'].dt.minute
dataframe.head(3)

,дата,год,месяц,день,час,минута
0,2001-01-07,2001,1,7,0,0
1,2001-01-14,2001,1,14,0,0
2,2001-01-21,2001,1,21,0,0


In [3]:
dataframe = pd.DataFrame()
dataframe['Прибыло'] = [pd.Timestamp('01-01-2017'), pd.Timestamp('01-04-2017')]
dataframe['Осталось'] = [pd.Timestamp('01-01-2017'), pd.Timestamp('01-06-2017')]
dataframe['Осталось'] - dataframe['Прибыло']

0   0 days
1   2 days
dtype: timedelta64[ns]

In [4]:
# Оставим только числовое значение
pd.Series(delta.days
          for delta in (dataframe['Осталось'] - dataframe['Прибыло']))

0    0
1    2
dtype: int64

### Кодирование дней недели
Дан вектор дат, и требуется узнать день недели для каждой даты

In [9]:
dates = pd.Series(pd.date_range("2/2/2002", periods=3, freq="ME"))
dates.dt.day_name()

0    Thursday
1      Sunday
2     Tuesday
dtype: object

In [10]:
dates.dt.weekday

0    3
1    6
2    1
dtype: int32

### Создание запаздывающего признака

In [12]:
dataframe = pd.DataFrame()
dataframe["даты"] = pd.date_range("1/1/2001", periods=5, freq='D')
dataframe["цена_акций"] = [1.2,2.2,3.3,4.4,5.5]
dataframe["цена_акций_в_предыдущий_день"] = dataframe["цена_акций"].shift(1)
dataframe

,даты,цена_акций,цена_акций_в_предыдущий_день
0,2001-01-01,1.2,NaN
1,2001-01-02,2.2,1.2
2,2001-01-03,3.3,2.2
3,2001-01-04,4.4,3.3
4,2001-01-05,5.5,4.4


### Обработка пропущенных дат во временном ряду

In [14]:
import numpy as np
time_index = pd.date_range("01/01/2010", periods=5, freq="ME")
dataframe = pd.DataFrame(index=time_index)
dataframe["продажи"] = [1.0,2.0,np.nan,np.nan,5.0]
dataframe.interpolate()

,продажи
2010-01-31,1.0
2010-02-28,2.0
2010-03-31,3.0
2010-04-30,4.0
2010-05-31,5.0


In [15]:
dataframe.ffill()

,продажи
2010-01-31,1.0
2010-02-28,2.0
2010-03-31,2.0
2010-04-30,2.0
2010-05-31,5.0


In [16]:
dataframe.bfill()

,продажи
2010-01-31,1.0
2010-02-28,2.0
2010-03-31,5.0
2010-04-30,5.0
2010-05-31,5.0


Интерполяция - метод заполнения промежутков, вызванных пропущенными значениями